In [ ]:
import boto3
import pandas as pd
from io import BytesIO

BUCKET = "online-retail-churn-siqi-dev"
REF_DATE = "2011-10-10"

SNAPSHOT_KEY = f"features/online_retail/dt=2026-01-12/customer_snapshot_ref={REF_DATE}.parquet"

session = boto3.Session(profile_name="siqi-dev")
s3 = session.client("s3")

obj = s3.get_object(Bucket=BUCKET, Key=SNAPSHOT_KEY)
snapshot = pd.read_parquet(BytesIO(obj["Body"].read()), engine="pyarrow")

snapshot.shape, snapshot.head()

: 